In [1]:
import torch
import numpy as np
import time
import torch.nn as nn
from torch.nn import functional as F

device = 'mps' if torch.backends.mps.is_available else 'cpu'
device = torch.device("cpu") # issue with mps and torch - mac

In [2]:
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

In [3]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
vocab_size

80

In [5]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
#     print(ix)
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

In [9]:
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[61, 58,  1, 55, 68, 78, 11,  0],
        [72, 61, 54, 71, 69,  9,  1, 56],
        [ 1, 73, 61, 58,  1, 73, 61, 71],
        [ 1, 55, 74, 62, 65, 73,  1, 73]])
targets:
tensor([[58,  1, 55, 68, 78, 11,  0,  0],
        [61, 54, 71, 69,  9,  1, 56, 65],
        [73, 61, 58,  1, 73, 61, 71, 68],
        [55, 74, 62, 65, 73,  1, 73, 61]])


In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B=batch size, T=sequence length, C=vocab size
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets )
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)            # Step 1: Get predictions from model
            logits = logits[:, -1, :]                     # Step 2: Take only the last time step's logits
            probs = F.softmax(logits, dim=-1)             # Step 3: Convert logits to probabilities
            index_next = torch.multinomial(probs, 1)      # Step 4: Pick next token based on probability
            index = torch.cat((index, index_next), 1)     # Step 5: Add it to the input
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


XR9[,K7li*T'Objz)a:ak9.JFg! 6ykKotu-*J9QiOe dDQL);NxVpRaUMrdB*tUSaP0CbTrt
99Q;_]r70gbqvZA,WhM'E[&9tFV")l54yh5GAf";j]wcn
r*T-N_pnKotKeIg(QOv,wp?Q['859I'w]f:drQ19t3madpDU
9Lk]2o4 u?WjsE; &G."'iSR,w.aXtR9tCvFHBA6Lbb
lI)Sx(hAOr"(gVr'HMjTaNRQFfeZakir6L7T ]ClF-1! 'm8cLP*'(Uc]&iig)qsev9_q&7s5AKxqot?MuQC6)JJ3]zIMJB:&vuZC5RQO_h,1v3-Hp0lHFxzXIpITTXyFVWMc3h*43as5q)yhRNDTz)T7aA&Twu"D;"lp0-1
,5YI85&FHw". x(.Jja
lI9t&"4WYTg.u:jcv3q:2m;Zh!YEyH
7R,BkN0-q1
l
Trpc:dJ]eM1nq:3"?3j]Ko&7FV dx6"?
7!6Y,sAp(i!eq5QAW,whF


In [17]:
# AdamW optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters ==0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    xb, yb = get_batch('train')
    
    # evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step: 0, train loss: 2.526, val loss: 2.549
step: 250, train loss: 2.498, val loss: 2.540
step: 500, train loss: 2.526, val loss: 2.550
step: 750, train loss: 2.524, val loss: 2.524
step: 1000, train loss: 2.515, val loss: 2.562
step: 1250, train loss: 2.493, val loss: 2.536
step: 1500, train loss: 2.534, val loss: 2.558
step: 1750, train loss: 2.525, val loss: 2.546
step: 2000, train loss: 2.519, val loss: 2.540
step: 2250, train loss: 2.504, val loss: 2.561
step: 2500, train loss: 2.496, val loss: 2.543
step: 2750, train loss: 2.497, val loss: 2.529
step: 3000, train loss: 2.487, val loss: 2.547
step: 3250, train loss: 2.517, val loss: 2.562
step: 3500, train loss: 2.489, val loss: 2.530
step: 3750, train loss: 2.497, val loss: 2.539
step: 4000, train loss: 2.476, val loss: 2.547
step: 4250, train loss: 2.478, val loss: 2.523
step: 4500, train loss: 2.485, val loss: 2.510
step: 4750, train loss: 2.501, val loss: 2.503
step: 5000, train loss: 2.492, val loss: 2.529
step: 5250, train l

In [ ]:
print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)